In [1]:
#%pip install git+https://github.com/ultralytics/ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
Setup complete  (16 CPUs, 31.9 GB RAM, 204.2/265.6 GB disk)


# Prunining

In [2]:
import torch
import os
from ultralytics import YOLO
import torch.nn.utils.prune as prune

# Define model paths
original_model_path = "Model/Model_M_best.pt"
pruned_model_path = "Model/Model_M_L1_Pruned.pt"

# Load model
model = YOLO(original_model_path)

# Identify Conv2d layers for pruning
parameters_to_prune = [(m, "weight") for _, m in model.model.named_modules() if isinstance(m, torch.nn.Conv2d)]

# Apply global unstructured pruning (L1 Norm, 51%)
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.51,
)

# Make pruning permanent
for module, param_name in parameters_to_prune:
    prune.remove(module, param_name)

# Save pruned model
model.save(pruned_model_path)

# Free up memory
del model
torch.cuda.empty_cache()

print(f"Pruned model saved at: {pruned_model_path}")

Pruned model saved at: Model/Model_M_L1_Pruned.pt


# Quantization 

 Install Dependencies

In [5]:
pip install ultralytics onnx onnxruntime-gpu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Export YOLO Model to ONNX

In [ ]:
from ultralytics import YOLO

# Load YOLO model
model = YOLO("Model/Model_M_L1_Pruned.pt")

# Define static input shape (batch_size=1, channels=3, height=640, width=640)
input_shape = (1, 3, 640, 640)

# Export to ONNX with fixed input shape
model.export(format="onnx", imgsz=640, dynamic=False, simplify=True)

print(" ONNX model exported successfully with fixed input shapes.")


Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CPU (AMD Ryzen 7 5700X 8-Core Processor)
YOLO11m summary (fused): 303 layers, 20,036,971 parameters, 0 gradients, 67.7 GFLOPs

PyTorch: starting from 'Model\Model_M_L1_Pruned.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 13, 8400) (38.6 MB)

ONNX: starting export with onnx 1.15.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success  1.8s, saved as 'Model\Model_M_L1_Pruned.onnx' (76.7 MB)

Export complete (2.6s)
Results saved to G:\Shahadath\Model
Predict:         yolo predict task=detect model=Model\Model_M_L1_Pruned.onnx imgsz=640  
Validate:        yolo val task=detect model=Model\Model_M_L1_Pruned.onnx imgsz=640 data=/kaggle/working/data.yaml  
Visualize:       https://netron.app
 ONNX model exported successfully with fixed input shapes.


Convert ONNX to TensorRT FP16 Engine by the Command Prompt 

trtexec --onnx=G:\Shahadath\Model\Model_M_L1_Pruned.onnx --saveEngine=G:\Shahadath\Model\Model_M_best_L1_Pruned_fp16.engine --fp16 --useSpinWait

Load and Run YOLO Model with TensorRT

In [12]:
from ultralytics import YOLO

# Load the optimized TensorRT model
model = YOLO("G:/Shahadath/Model/Model_M_best.pt")

# Run inference on an image
results = model("G:/Shahadath/Dataset/test/images/5096_0745204059_03_WRI-R1_F010.png", verbose=False)

# Display results
print(results)


[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'boneanomaly', 1: 'bonelesion', 2: 'foreignbody', 3: 'fracture', 4: 'metal', 5: 'periostealreaction', 6: 'pronatorsign', 7: 'softtissue', 8: 'text'}
obb: None
orig_img: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
       

In [ ]:
from ultralytics import YOLO

# Load the optimized TensorRT model
model = YOLO("G:/Shahadath/Model/Model_M_best_L1_Pruned_fp16.engine")

# Run inference on an image
results = model("G:/Shahadath/Dataset/test/images/5096_0745204059_03_WRI-R1_F010.png", verbose=False)

# Display results
print(results)


WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading G:\Shahadath\Model\Model_M_best_L1_Pruned_fp16.engine for TensorRT inference...
WARNING  Metadata not found for 'model=G:/Shahadath/Model/Model_M_best_L1_Pruned_fp16.engine'
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'class0', 1: 'class1', 2: 'class2', 3: 'class3', 4: 'class4', 5: 'class5', 6: 'class6', 7: 'class7', 8: 'class8', 9: 'class9', 10: 'class10', 11: 'class11', 12: 'class12', 13: 'class13', 14: 'class14', 15: 'class15', 16: 'class16', 17: 'class17', 18: 'class18', 19: 'class19', 20: 'class20', 21: 'class21', 22: 'class22', 23: 'class23', 24: 'class24', 25: 'class25', 26: 'class26', 27: 'class27', 28: 'class28', 29: 'class29', 30: 'class30', 31: 'class31', 32: 'class32', 33: 'class33', 34: 'class3

Benchmark Performance (Before vs After TensorRT)

In [ ]:
import time
import torch
import os
from ultralytics import YOLO
import pandas as pd

# Define paths
pytorch_model_path = "G:/Shahadath/Model/Model_M_best.pt"  # Original PyTorch model
tensorrt_model_path = "G:/Shahadath/Model/Model_M_best_L1_Pruned_fp16.engine"  # Optimized TensorRT FP16 model
test_images_dir = "G:/Shahadath/Dataset/test/images"  # Path to test images

# Get list of test images
img_files = [f for f in os.listdir(test_images_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
num_images = len(img_files)

# ------------------ Function to Benchmark Model ------------------
def benchmark_model(model, model_name):
    """Runs inference on all test images and calculates inference time statistics."""
    inference_times = []

    for img_file in img_files:
        img_path = os.path.join(test_images_dir, img_file)

        with torch.no_grad():
            start_time = time.time()
            _ = model(img_path, verbose=False)  # Run inference
            inference_time = (time.time() - start_time) * 1000  # Convert to milliseconds

        inference_times.append(inference_time)

    # Compute mean, min, and max inference times
    mean_time = sum(inference_times) / len(inference_times)
    min_time = min(inference_times)
    max_time = max(inference_times)

    print(f" {model_name} - Mean: {mean_time:.2f} ms | Min: {min_time:.2f} ms | Max: {max_time:.2f} ms")

    return mean_time, min_time, max_time

# ------------------ Load Models ------------------
print("\n🔹 Loading PyTorch Model...")
pytorch_model = YOLO(pytorch_model_path)

print("\n🔹 Loading TensorRT FP16 Model...")
tensorrt_model = YOLO(tensorrt_model_path)

# ------------------ Run Benchmark ------------------
print("\n🚀 Running Benchmark...")

# PyTorch Benchmark
pytorch_mean, pytorch_min, pytorch_max = benchmark_model(pytorch_model, "PyTorch Model")

# TensorRT FP16 Benchmark
tensorrt_mean, tensorrt_min, tensorrt_max = benchmark_model(tensorrt_model, "TensorRT FP16 Model")

# ------------------ Calculate Speedup ------------------
speedup_factor = pytorch_mean / tensorrt_mean
print(f"\n🔥 Speedup with TensorRT FP16: {speedup_factor:.2f}x Faster 🚀")

# ------------------ Display Results in Table ------------------
results = pd.DataFrame({
    "Model": ["PyTorch", "TensorRT FP16"],
    "Avg Inference Time (ms)": [pytorch_mean, tensorrt_mean],
    "Speedup Factor": ["-", f"{speedup_factor:.2f}x"]
})

print("\n📊 Benchmark Results:")
print(results.to_string(index=False))

# --------------------- Performance Comparison ---------------------
df = pd.DataFrame({
    "Metric": ["Mean Inference Time (ms)", "Min Inference Time (ms)", "Max Inference Time (ms)", "Speedup Factor"],
    "PyTorch": [round(pytorch_mean, 2), round(pytorch_min, 2), round(pytorch_max, 2), "-"],
    "TensorRT FP16": [round(tensorrt_mean, 2), round(tensorrt_min, 2), round(tensorrt_max, 2),
                      round(speedup_factor, 2)]
})

print("\n📊 Performance Comparison (PyTorch vs TensorRT FP16):\n")
print(df.to_string(index=False))



🔹 Loading PyTorch Model...

🔹 Loading TensorRT FP16 Model...
WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.

🚀 Running Benchmark...
✅ PyTorch Model - Mean: 23.11 ms | Min: 15.62 ms | Max: 467.21 ms
Loading G:\Shahadath\Model\Model_M_best_L1_Pruned_fp16.engine for TensorRT inference...
WARNING  Metadata not found for 'model=G:/Shahadath/Model/Model_M_best_L1_Pruned_fp16.engine'
✅ TensorRT FP16 Model - Mean: 16.15 ms | Min: 9.09 ms | Max: 305.39 ms

🔥 Speedup with TensorRT FP16: 1.43x Faster 🚀

📊 Benchmark Results:
        Model  Avg Inference Time (ms) Speedup Factor
      PyTorch                23.105781              -
TensorRT FP16                16.146854          1.43x

📊 Performance Comparison (PyTorch vs TensorRT FP16):

                  Metric PyTorch  TensorRT FP16
Mean Inference Time (ms)   23.11          16.15
 Min Inference Time (ms)   15.62         

In [1]:
import time
import torch
import os
import pandas as pd
from ultralytics import YOLO

# Define the models and their paths
models_info = [
    {
        "name": "Model_M_best.pt",
        "weights_path": "Model/Model_M_best.pt"
    },
    {
        "name": "Model_M_best_L1_Pruned_fp16",
        "weights_path": "G:/Shahadath/Model/Model_M_best_L1_Pruned_fp16.engine"
    }
]

# Path to the test images
test_images_dir = 'Dataset/test/images'

# Path to data.yaml
data_yaml_path = 'data.yaml'

# Get list of image files in the test directory
img_files = [f for f in os.listdir(test_images_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
num_images = len(img_files)

# Initialize a list to store results
results = []
inference_times = {}  # Store inference times for speedup calculation

# Iterate over each model and evaluate
for model_info in models_info:
    model_name = model_info["name"]
    weights_path = model_info["weights_path"]
    
    # Load the YOLO model
    model = YOLO(weights_path)
    
    # -------------------- Model Evaluation --------------------
    # Evaluate the model on the test set
    metrics = model.val(data=data_yaml_path, split="test", batch=16, imgsz=640)

    # Extract metrics safely
    try:
        precision = metrics.box.p.mean()  # Mean Precision
        recall = metrics.box.r.mean()     # Mean Recall
        map50 = metrics.box.map50.mean()  # Mean mAP@50
        map = metrics.box.map.mean()      # Mean mAP@50-95
    except AttributeError:
        print(f"Error extracting metrics for {model_name}. Skipping...")
        continue

    # -------------------- Inference Time Calculation --------------------
    inference_times_list = []  # Store inference times for min/max calculation

    # Loop through each image file and perform inference
    for img_file in img_files:
        img_path = os.path.join(test_images_dir, img_file)

        # Suppress output from YOLO during inference
        with torch.no_grad():
            start_time = time.time()
            _ = model(img_path, verbose=False)  # Perform inference with verbose=False
            inference_time = (time.time() - start_time) * 1000  # Convert to milliseconds

        inference_times_list.append(inference_time)

    # Compute min, max, and mean inference times
    avg_inference_time = sum(inference_times_list) / num_images
    min_inference_time = min(inference_times_list)
    max_inference_time = max(inference_times_list)

    # Store inference times for speedup calculation
    inference_times[model_name] = avg_inference_time

    # -------------------- Model Parameters and FLOPs --------------------
    # Extracting model parameters (in millions)
    try:
        params = sum(p.numel() for p in model.parameters()) / 1e6  # In millions
        flops = params * 2  # Rough estimate of FLOPs (billions)
    except Exception as e:
        print(f"Error calculating parameters/FLOPs for {model_name}: {e}")
        params, flops = 0, 0  # Set to 0 in case of an error

    # Use "N/A" if params or flops are zero
    params_str = "N/A" if params == 0 else round(params, 3)
    flops_str = "N/A" if flops == 0 else round(flops, 3)

    # Append the results for this model
    results.append({
        "Model": model_name,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "mAP@50": round(map50, 3),
        "mAP@50-95": round(map, 3),
        "Params(M)": params_str,
        "FLOPs(B)": flops_str,
        "Min Inference Time (ms)": round(min_inference_time, 2),
        "Max Inference Time (ms)": round(max_inference_time, 2),
        "Mean Inference Time (ms)": round(avg_inference_time, 2)
    })

# ------------------ Calculate Speedup ------------------
try:
    pytorch_mean = inference_times["Model_M_best.pt"]
    tensorrt_mean = inference_times["Model_M_best_L1_Pruned_fp16"]
    speedup_factor = pytorch_mean / tensorrt_mean
    speedup_str = f"{speedup_factor:.2f}x"
except KeyError:
    speedup_str = "N/A"

# ------------------ Display Results ------------------
df = pd.DataFrame(results)

print("\n Model Evaluation Results:\n")
print(df.to_string(index=False))

# --------------------- Performance Comparison ---------------------
df_comparison = pd.DataFrame({
    "Metric": ["Mean Inference Time (ms)", "Min Inference Time (ms)", "Max Inference Time (ms)", "Speedup Factor"],
    "PyTorch": [round(pytorch_mean, 2), round(min_inference_time, 2), round(max_inference_time, 2), "-"],
    "TensorRT FP16": [round(tensorrt_mean, 2), round(min_inference_time, 2), round(max_inference_time, 2), speedup_str]
})

print("\n Performance Comparison (PyTorch vs TensorRT FP16):\n")
print(df_comparison.to_string(index=False))


Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
YOLO11m summary (fused): 303 layers, 20,036,971 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:15<00:00,  8.45it/s]


                   all       2033       4720      0.843       0.68      0.836       0.57
           boneanomaly         16         19       0.81      0.526      0.615      0.431
            bonelesion          8          8      0.926      0.625      0.735      0.463
           foreignbody          1          1          1          0      0.995      0.597
              fracture       1351       1784      0.931       0.95      0.977      0.666
                 metal         74         85       0.96          1      0.995      0.884
    periostealreaction        230        352      0.752      0.699      0.775      0.439
          pronatorsign         47         47      0.699      0.745      0.818      0.508
            softtissue         38         39      0.533       0.59      0.619      0.392
                  text       2029       2385      0.976      0.987      0.994      0.755
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect

val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2033/2033 [00:17<00:00, 114.08it/s]


                   all       2033       4720      0.843      0.658      0.837      0.571
           boneanomaly         16         19      0.751      0.477      0.605      0.427
            bonelesion          8          8      0.867        0.5      0.716      0.466
           foreignbody          1          1          1          0      0.995      0.597
              fracture       1351       1784      0.936      0.948      0.976      0.666
                 metal         74         85      0.942          1      0.995       0.88
    periostealreaction        230        352      0.777      0.682      0.778      0.439
          pronatorsign         47         47      0.768      0.766      0.816      0.505
            softtissue         38         39      0.566      0.569      0.659      0.408
                  text       2029       2385      0.977      0.984      0.993      0.753
Speed: 0.3ms preprocess, 3.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect

In [7]:
pip install onnx onnxruntime thop

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
pip install onnx onnxruntime torch ptflops


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
pip install onnx onnx2torch ptflops torch torchvision


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/81.0 kB ? eta -:--:--
   ---------------------------------------- 81.0/81.0 kB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
from thop import profile
from ultralytics import YOLO

# Load YOLO model
model = YOLO("Model/Model_M_L1_Pruned.pt")

# Define static input shape (batch_size=1, channels=3, height=640, width=640)
input_shape = (1, 3, 640, 640)
dummy_input = torch.randn(input_shape)

# Get the PyTorch model from YOLO
pytorch_model = model.model  # Extract underlying PyTorch model

# Compute FLOPs and Parameters
flops, params = profile(pytorch_model, inputs=(dummy_input,))

# Print results
print(f"Total Parameters: {params / 1e6:.3f}M")  # Convert to Million (M)
print(f"Total FLOPs: {flops / 1e9:.3f}G")  # Convert to Giga (G)


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
Total Parameters: 20.060M
Total FLOPs: 34.112G


In [4]:
import torch
from ultralytics import YOLO

# Paths to required files
weights_path = 'G:/Shahadath/Model/Model_M_best_L1_Pruned_fp16.engine'
data_yaml_path = 'data.yaml'

# Load the YOLO model
model = YOLO(weights_path)

# Evaluate the model on the test set
metrics = model.val(data=data_yaml_path, split="test", batch=16, imgsz=640)

# Extract and print metrics
precision = metrics.box.p.mean()  # Mean Precision
recall = metrics.box.r.mean()     # Mean Recall
map50 = metrics.box.map50.mean()  # Mean mAP@50
map = metrics.box.map.mean()      # Mean mAP@50-95

print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"mAP@50: {round(map50, 3)}")
print(f"mAP@50-95: {round(map, 3)}")

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
Loading G:\Shahadath\Model\Model_M_best_L1_Pruned_fp16.engine for TensorRT inference...
WARNING  Metadata not found for 'model=G:/Shahadath/Model/Model_M_best_L1_Pruned_fp16.engine'


val: Scanning G:\Shahadath\Dataset\test\labels.cache... 2033 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2033/2033 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2033/2033 [00:16<00:00, 121.94it/s]


                   all       2033       4720      0.843      0.658      0.837      0.571
           boneanomaly         16         19      0.751      0.477      0.605      0.427
            bonelesion          8          8      0.867        0.5      0.716      0.466
           foreignbody          1          1          1          0      0.995      0.597
              fracture       1351       1784      0.936      0.948      0.976      0.666
                 metal         74         85      0.942          1      0.995       0.88
    periostealreaction        230        352      0.777      0.682      0.778      0.439
          pronatorsign         47         47      0.768      0.766      0.816      0.505
            softtissue         38         39      0.566      0.569      0.659      0.408
                  text       2029       2385      0.977      0.984      0.993      0.753
Speed: 0.3ms preprocess, 3.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect

In [3]:
import torch
from ultralytics import YOLO

# Paths to required files
weights_path = 'G:/Shahadath/Model/Model_M_best_L1_Pruned_fp16.engine'
data_yaml_path = 'data.yaml'

# Load the YOLO model
model = YOLO(weights_path)

# Evaluate the model on the val set
metrics = model.val(data=data_yaml_path, split="val", batch=16, imgsz=640)

# Extract and print metrics
precision = metrics.box.p.mean()  # Mean Precision
recall = metrics.box.r.mean()     # Mean Recall
map50 = metrics.box.map50.mean()  # Mean mAP@50
map = metrics.box.map.mean()      # Mean mAP@50-95

print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"mAP@50: {round(map50, 3)}")
print(f"mAP@50-95: {round(map, 3)}")

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Ultralytics 8.3.50  Python-3.9.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)
Loading G:\Shahadath\Model\Model_M_best_L1_Pruned_fp16.engine for TensorRT inference...
WARNING  Metadata not found for 'model=G:/Shahadath/Model/Model_M_best_L1_Pruned_fp16.engine'


val: Scanning G:\Shahadath\Dataset\val\labels.cache... 4066 images, 4 backgrounds, 0 corrupt: 100%|██████████| 4066/4066 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4066/4066 [00:35<00:00, 113.10it/s]


                   all       4066       9500      0.808      0.755       0.83      0.584
           boneanomaly         46         71      0.694      0.549      0.634      0.381
            bonelesion          9          9      0.728      0.556      0.766      0.462
           foreignbody          4          4       0.85          1      0.995      0.858
              fracture       2736       3653      0.945       0.94       0.98      0.663
                 metal        128        148      0.954      0.966      0.991      0.873
    periostealreaction        421        660      0.787      0.673      0.777      0.435
          pronatorsign        125        126      0.762      0.627      0.778      0.482
            softtissue         72         77      0.575      0.506      0.552      0.345
                  text       4055       4752      0.976      0.979      0.993      0.754
Speed: 0.3ms preprocess, 3.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect